# 00 | Project Overview

## Objective
Build a **Notebook-First Lakehouse** that behaves like a production-ready analytics project:
- reproducible runs
- medallion contracts
- SQL marts in dbt
- quality gates with Great Expectations
- observability metrics for each stage

## Run Parameters
These notebooks are Papermill-compatible and accept:
- `source`
- `dataset`
- `run_date`
- `force_refresh`

In [ ]:
# Parameters
source = "fivethirtyeight"
dataset = "recent_grads,bechdel_movies"
run_date = "2026-02-22"
force_refresh = False

In [ ]:
import sys
from pathlib import Path

ROOT_DIR = Path.cwd()
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

from datetime import datetime, timezone

from src.common.io import update_pipeline_metrics
from src.common.paths import (
    BRONZE_DIR,
    GOLD_DIR,
    RAW_DATA_DIR,
    SILVER_DIR,
    ensure_project_dirs,
)

ensure_project_dirs()
selected_datasets = [item.strip() for item in dataset.split(',') if item.strip()]

update_pipeline_metrics(
    {
        'source': source,
        'datasets': selected_datasets,
        'run_date': run_date,
        'pipeline_started_at': datetime.now(timezone.utc).replace(microsecond=0).isoformat(),
    }
)

print('Run date:', run_date)
print('Source:', source)
print('Datasets:', selected_datasets)
print('Raw path:', RAW_DATA_DIR)
print('Bronze path:', BRONZE_DIR)
print('Silver path:', SILVER_DIR)
print('Gold path:', GOLD_DIR)

## Medallion Definition
- **Bronze**: standardized raw payload + ingestion metadata (`_ingested_at`, `_source`, `_dataset`, `_file_path`, `_row_hash`, `_batch_id`).
- **Silver**: typed + deduplicated + key-safe datasets.
- **Gold**: curated marts and KPI outputs from dbt.